In [2]:
#Georgios Cheirmpos - 3130230
#Machine Learning Project 1
#PROJECT DIRECTORY MUST BE LIKE BELOW
#cifar-10-batches-py/<contents of cifar zip>
#mnistdata/<mnistfiles>
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.utils
import itertools

In [3]:
def load_data_mnist():
    df = None  
    y_train = []
    for i in range( 10 ):
        tmp = pd.read_csv( 'mnistdata/train%d.txt' % i, header=None, sep=" " )
        #build labels - one hot vector
        
        hot_vector = [ 1 if j == i else 0 for j in range(0,10) ]
        
        for j in range( tmp.shape[0] ):
            y_train.append( hot_vector )
        #concatenate dataframes by rows    
        if i == 0:
            df = tmp
        else:
            df = pd.concat( [df, tmp] )
    train_data = df.as_matrix()
    y_train = np.array( y_train )    
    #load test files
    df = None    
    y_test = []
    for i in range( 10 ):
        tmp = pd.read_csv( 'mnistdata/test%d.txt' % i, header=None, sep=" " )        
        hot_vector = [ 1 if j == i else 0 for j in range(0,10) ]       
        for j in range( tmp.shape[0] ):
            y_test.append( hot_vector )
        #concatenate dataframes by rows    
        if i == 0:
            df = tmp
        else:
            df = pd.concat( [df, tmp] )
    test_data = df.as_matrix()
    y_test = np.array( y_test )    
    return train_data, test_data, y_train, y_test

def load_data_cifar():
    y_train = []
    X_train = None
    for i in range( 1,6 ):
        tmp = pd.read_pickle('cifar-10-batches-py/data_batch_%d' % i)
        if i == 1 :
            X_train = tmp['data']    
        else:
            X_train = np.vstack((X_train, tmp['data']))

        for k in range(len(tmp['labels'])):
            hot_vector = [ 1 if j == tmp['labels'][k] else 0 for j in range(0,10) ]
            y_train.append(hot_vector)
        #concatenate dataframes by rows    
    y_train = np.array( y_train )
    #load test files
    X_test = None    
    y_test = []
    tmp = pd.read_pickle('cifar-10-batches-py/test_batch')
    #build labels - one hot vector
    X_test = tmp['data'] 
    for k in range(len(tmp['labels'])):
        hot_vector = [ 1 if j == tmp['labels'][k] else 0 for j in range(0,10) ]
        y_test.append(hot_vector)
    #concatenate dataframes by rows    
    y_test = np.array( y_test )    
    return X_train, X_test, y_train, y_test

In [4]:
#gray preprocess for image
def grayscale_cifar(X_train, X_test):
    #gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
    X_train_g = []
    X_test_g = []
    for image in X_train:
        X_train_g.append(np.dot(image.reshape(32, 32, 3), [0.299, 0.587, 0.114]).reshape(32*32,).astype(float)/255)
    for image in X_test:
        X_test_g.append(np.dot(image.reshape(32, 32, 3), [0.299, 0.587, 0.114]).reshape(32*32,).astype(float)/255)
    return np.array(X_train_g), np.array(X_test_g)

In [5]:
def load_data(which_one):
    if which_one == 'mnist':
        X_train, X_test, y_train, y_test = load_data_mnist()
        #normalize
        X_train = X_train.astype(float)/255
        X_test = X_test.astype(float)/255       
        #add bias vector
        X_train = np.hstack( (np.ones((X_train.shape[0],1) ), X_train) )
        X_test = np.hstack( (np.ones((X_test.shape[0],1) ), X_test) )
        return X_train, X_test, y_train, y_test
    elif which_one == 'cifar':
        X_train, X_test, y_train, y_test = load_data_cifar()
        #preprocess cifar
        X_train, X_test = grayscale_cifar(X_train, X_test)
        #add bias
        X_train = np.hstack( (np.ones((X_train.shape[0],1) ), X_train) )
        X_test = np.hstack( (np.ones((X_test.shape[0],1) ), X_test) )
        return X_train, X_test, y_train, y_test
    

In [6]:
def gradcheck_softmax(Winit, X, t, lamda):
    
    W = np.random.rand(*Winit.shape)
    epsilon = 1e-6
    
    _list = np.random.randint(X.shape[0], size=5)
    x_sample = np.array(X[_list, :])
    t_sample = np.array(t[_list, :])
    
    Ew, gradEw = cost_grad_softmax(W, x_sample, t_sample, lamda)
    
    
    numericalGrad = np.zeros(gradEw.shape)
    # Compute all numerical gradient estimates and store them in
    # the matrix numericalGrad
    for k in range(numericalGrad.shape[0]):
        for d in range(numericalGrad.shape[1]):
            
            #add epsilon to the w[k,d]
            w_tmp = np.copy(W)
            w_tmp[k, d] += epsilon
            e_plus, _ = cost_grad_softmax(w_tmp, x_sample, t_sample, lamda)
            
            #subtract epsilon to the w[k,d]
            w_tmp = np.copy(W)
            w_tmp[k, d] -= epsilon
            e_minus, _ = cost_grad_softmax( w_tmp, x_sample, t_sample, lamda)
            
            #approximate gradient ( E[ w[k,d] + theta ] - E[ w[k,d] - theta ] ) / 2*e
            numericalGrad[k, d] = (e_plus - e_minus) / (2 * epsilon)
    
    return ( gradEw, numericalGrad )


def gradcheck_activation(hidden_layer_weights, output_layer_weights, output_values_batch, X_train_mini, y_train_mini, activation, lamda, eta):

    W = np.random.rand(*hidden_layer_weights.shape)
    epsilon = 1e-6
    gradEw1 = cost_grad_activation(hidden_layer_weights, output_layer_weights, output_values_batch, X_train_mini, y_train_mini, activation, lamda, eta)
    #precalculate new weights
    hidden_layer_weights = hidden_layer_weights + eta * gradEw1      
    _list = np.random.randint(X_train_mini.shape[0], size=5)
    x_sample = np.array(X_train_mini[_list, :])
    t_sample = np.array(y_train_mini[_list, :])
    numericalGrad = np.zeros(gradEw1.shape)
    # Compute all numerical gradient estimates and store them in
    # the matrix numericalGrad
    for k in range(numericalGrad.shape[0]):
        if k % 10 == 0:
            print(k)
        #pass new hidden activated functions to cost_grad_softmax
        #to check changes in LOSS
        for d in range(numericalGrad.shape[1]):
           
            #add epsilon to the w[k,d]
            w_tmp = np.copy(W)
            w_tmp[k, d] += epsilon  
            #W1 * X
            hidden_values_batch = x_sample.dot((w_tmp+eta*gradEw1).T)
            hidden_values_batch_activated = activation_func(activation, hidden_values_batch)
            #add bias for output
            hidden_values_batch_activated = np.hstack( (np.ones((hidden_values_batch_activated.shape[0],1) ), hidden_values_batch_activated))
            e_plus, _ = cost_grad_softmax(output_layer_weights, hidden_values_batch_activated, t_sample, lamda)
           
            #subtract epsilon to the w[k,d]
            w_tmp = np.copy(W)
            w_tmp[k, d] -= epsilon
            #W1 * X
            hidden_values_batch = x_sample.dot((w_tmp+eta*gradEw1).T)
            hidden_values_batch_activated = activation_func(activation, hidden_values_batch)
            #add bias for output
            hidden_values_batch_activated = np.hstack( (np.ones((hidden_values_batch_activated.shape[0],1) ), hidden_values_batch_activated))
            e_minus, _ = cost_grad_softmax(output_layer_weights, hidden_values_batch_activated, t_sample, lamda)
            
            #approximate gradient ( E[ w[k,d] + theta ] - E[ w[k,d] - theta ] ) / 2*e
            numericalGrad[k, d] = (e_plus - e_minus) / (2 * epsilon)
    
            #numericalGrad[k, d] = (sum(sum(e_plus)) - sum(sum(e_minus))) / (2 * epsilon)
    
    return ( gradEw1, numericalGrad )

In [7]:
#softmax activation
def softmax( x, ax=1 ):
    m = np.max( x, axis=ax, keepdims=True )#max per row
    p = np.exp( x - m )
    return ( p / np.sum(p,axis=ax,keepdims=True) )

#softmax derivative
def softmax_der(x, ax):
    soft = softmax(x, ax)
    return soft*(1 - soft)

#activation functions
def activation_func(func, a):
    if func == 0:
        return np.log(1+np.exp(a))
    elif func == 1:
        return (np.exp(a) - np.exp(-a))/(np.exp(a) + np.exp(-a))
    elif func == 2:   
        return np.cos(a)
    else:
        exit("Undefined function")

#derivatives of activation functions
def activation_func_der(func, a):
    if func == 0:
        return np.exp(a) / (1 + np.exp(a))
    elif func == 1:
        return 1 - (activation_func(1, a))**2
    elif func == 2:   
        return -np.sin(a)
    else:
        exit("Undefined function")
        

def cost_grad_softmax(output_layer_weights, hidden_values_batch_activated, y_true, lamda):
    output_values_batch = hidden_values_batch_activated.dot(output_layer_weights.T)    
    output_values_batch_activated = softmax(output_values_batch)
    max_error = np.max(output_values_batch, axis=1)
    # Compute the cost function to check convergence
    # Using the logsumexp trick for numerical stability - lec8.pdf slide 43
    Ew2 = np.sum(y_true * output_values_batch) - np.sum(max_error) - \
        np.sum(np.log(np.sum(np.exp( \
                output_values_batch - np.array([max_error, ] * output_values_batch.shape[1]).T), 1))) - \
        (0.5 * lamda) * np.sum(np.square(output_layer_weights))

    # calculate gradient
    gradEw2 = (y_true - output_values_batch_activated).T.dot(hidden_values_batch_activated) - lamda * output_layer_weights
    return Ew2, gradEw2

def cost_grad_activation(hidden_layer_weights, output_layer_weights, output_values_batch, X_train_mini, y_train_mini, activation, lamda, eta):
    #activation output
    output_values_batch_activated = softmax(output_values_batch)
    #derivative activation hidden
    derivative_activ_hidden = activation_func_der(activation, hidden_layer_weights.dot(X_train_mini.T)) 
    #derivative Cost partial derivative
    der_cost_part = (y_train_mini - output_values_batch_activated).dot(output_layer_weights) 
    #W1 gradient
    gradEw1 = (derivative_activ_hidden*der_cost_part.T[:-1]).dot(X_train_mini)
    return gradEw1

In [8]:
#initialize weights. Divide by 1000 for stability
def init_weights(hidden_nodes, dimensions, classes):
    hidden_layer_weights = np.random.rand(hidden_nodes,dimensions)/1000
    output_layer_weights = np.random.rand(classes, hidden_nodes+1)/1000
    return hidden_layer_weights, output_layer_weights


def predict(W1, W2, X_test, activation):
    # W1 * X
    hidden = X_test.dot(W1.T)
    hidden = activation_func(activation, hidden)
    #Add bias for output
    hidden = np.hstack( (np.ones((hidden.shape[0],1) ), hidden) )
    # W2 * Output_of_hidden
    ytest = softmax( hidden.dot(W2.T) )
    ttest = np.argmax(ytest, 1)
    print(np.mean( ttest == np.argmax(y_test,1) ))

def train(X_train, y_train, options):
    #get the options and init weights
    lamda, hidden_nodes, activation, eta, batch_size, epochs, verbose, grad_check = options.values()
    hidden_layer_weights, output_layer_weights = init_weights(hidden_nodes, X_train.shape[1], y_train.shape[1])
    
    for epoch in range(epochs):
        #loop by batch
        for i in range(0, X_train.shape[0], batch_size):
            X_train_mini = X_train[i:i + batch_size]
            y_train_mini = y_train[i:i + batch_size]
            
            #W1 * X
            hidden_values_batch = X_train_mini.dot(hidden_layer_weights.T)
            hidden_values_batch_activated = activation_func(activation, hidden_values_batch)
            #add bias for output
            hidden_values_batch_activated = np.hstack( (np.ones((hidden_values_batch_activated.shape[0],1) ), hidden_values_batch_activated) )

            #W2 * output_of_hidden
            output_values_batch = hidden_values_batch_activated.dot(output_layer_weights.T)    
            output_values_batch_activated = softmax(output_values_batch)
        
            #calculate gradients
            Ew2, gradEw2 = cost_grad_softmax(output_layer_weights, hidden_values_batch_activated, y_train_mini, lamda)           
            gradEw1 = cost_grad_activation(hidden_layer_weights, output_layer_weights, output_values_batch, X_train_mini, y_train_mini, activation, lamda, eta)
            
            if verbose:
                if i % 30000 == 0:
                    print('Epoch:' + str(epoch+1), 'Iteration : %d, Cost function :%f' % (i, Ew2))
                #slow
                if grad_check:
                    print( "gradEw2 shape: ", gradEw2.shape )                 
                    x,y =  gradcheck_softmax(output_layer_weights, hidden_values_batch_activated, y_train_mini, lamda)
                    print( "The difference estimate for gradient of W2 is : ", np.max(np.abs(x-y)))
                    #slow..SLOW!!!!!!!!
                    print( "gradEw1 shape: ", gradEw1.shape )  
                    x,y =  gradcheck_activation(hidden_layer_weights, output_layer_weights, output_values_batch, X_train_mini, y_train_mini, activation, lamda, eta)
                    print( "The difference estimate for gradient of W1 is : ", np.max(np.abs(x-y)))
            
            #updates weights
            output_layer_weights = output_layer_weights + eta * gradEw2
            hidden_layer_weights = hidden_layer_weights + eta * gradEw1
            
             
        if verbose == 0:
            print('Epoch:' + str(epoch))
    return hidden_layer_weights, output_layer_weights



In [34]:
#uncomment to load dataset
#X_train, X_test, y_train, y_test = load_data('cifar')
X_train, X_test, y_train, y_test = load_data('mnist')    
X_train, y_train = sklearn.utils.shuffle(X_train, y_train) 
#set parameters 
options = {'lamda': .1,
           'hidden_nodes': 200,           
           'activation':0,
           'eta': 1e-4,
           'batch_size': 200, 
           'epochs': 5, 
           'verbose': 0,
           'grad_check': 0}
W1, W2 = train(X_train, y_train, options)
predict(W1, W2, X_test, options['activation'])



Epoch:0
Epoch:1
Epoch:2
Epoch:3
Epoch:4
Epoch:5
Epoch:6
Epoch:7
Epoch:8
Epoch:9
Epoch:10
Epoch:11
Epoch:12
Epoch:13
Epoch:14
Epoch:15
Epoch:16
Epoch:17
Epoch:18
Epoch:19
Epoch:20
Epoch:21
Epoch:22
Epoch:23
Epoch:24
Epoch:25
Epoch:26
Epoch:27
Epoch:28
Epoch:29
Epoch:30
Epoch:31
Epoch:32
Epoch:33
Epoch:34
Epoch:35
Epoch:36
Epoch:37
Epoch:38
Epoch:39
Epoch:40
Epoch:41
Epoch:42
Epoch:43
Epoch:44
Epoch:45
Epoch:46
Epoch:47
Epoch:48
Epoch:49
Epoch:50
Epoch:51
Epoch:52
Epoch:53
Epoch:54
Epoch:55
Epoch:56
Epoch:57
Epoch:58
Epoch:59
0.4051


In [9]:
''' TEST BENCH CROSS_VALIDATION
lamda = [.001, .002, .005, .01, .02, .05, .1, .2, .5,]# 1, 2, 5]
epochs = [1, 2, 5, 10, 20]
eta = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2]#, 1e-1, 1]
batch_size = [100, 200, 300]
hidden_nodes = [ 100, 200]
activations = [1, 2]
data =  ['cifar'] #['mnist',

for i in range(len(data)):
    X_train, X_test, y_train, y_test = load_data(data[i])    
    X_train, y_train = sklearn.utils.shuffle(X_train, y_train)
    for activation in activations:
        for h_nodes in hidden_nodes:
            for lamd in lamda:
                for et in eta:
                    for epoch in epochs:
                        options = {'lamda': lamd, 'hidden_nodes': h_nodes, 'activation':activation, 'eta': et, 'batch_size': h_nodes, 
                                   'epochs': epoch, 'verbose': 2, 'grad_check': 0}
                        print(options)
                        W1, W2 = train(X_train, y_train, options)
                        predict(W1, W2, X_test, options['activation'])
'''

" TEST BENCH CROSS_VALIDATION\nlamda = [.001, .002, .005, .01, .02, .05, .1, .2, .5,]# 1, 2, 5]\nepochs = [1, 2, 5, 10, 20]\neta = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2]#, 1e-1, 1]\nbatch_size = [100, 200, 300]\nhidden_nodes = [ 100, 200]\nactivations = [1, 2]\ndata =  ['cifar'] #['mnist',\n\nfor i in range(len(data)):\n    X_train, X_test, y_train, y_test = load_data(data[i])    \n    X_train, y_train = sklearn.utils.shuffle(X_train, y_train)\n    for activation in activations:\n        for h_nodes in hidden_nodes:\n            for lamd in lamda:\n                for et in eta:\n                    for epoch in epochs:\n                        options = {'lamda': lamd, 'hidden_nodes': h_nodes, 'activation':activation, 'eta': et, 'batch_size': h_nodes, \n                                   'epochs': epoch, 'verbose': 2, 'grad_check': 0}\n                        print(options)\n                        W1, W2 = train(X_train, y_train, options)\n                        predict(W1, W2, X_test, opti